### Merge obj det and determiner label files

In [60]:
import os 
import cv2
import pandas as pd 
import math
from collections import defaultdict
import inflection as inf
import json

#######################################################################################
############################# CHANGE FILENAMES HERE ###################################
#######################################################################################

ann_dir = "../../annotations/real/smallv1"
objdet_filename = "iccv-real-small-diffobj-objdet-1.json"
det_filename = "iccv-real-small-diffobj-detlabels-1.json"
orig_annotations_filepath = "../../annotations/real/smallv1/categories-diffobj.json"
save_filename = "merged.json"

objdet_json = json.load(open(os.path.join(ann_dir, objdet_filename)))
det_json = json.load(open(os.path.join(ann_dir, det_filename)))
orig_json = json.load(open(orig_annotations_filepath))

categories = orig_json["categories"]

orig_cat_map = {}

for cat in categories:
    orig_cat_map[cat["name"]] = cat["id"]

images = []
objdet_images = objdet_json["images"]
det_images = det_json["images"]
det_categories = det_json["categories"]
oracle_annotations = objdet_json["annotations"]

cat_map = {}
id_map = {}
counts = defaultdict(int)

for i, cat in enumerate(det_categories):
    cat_map[cat["id"]] = cat["name"]

for i, img in enumerate(det_images): 
    img = img.copy() 
    image_id = img["id"]
    img["file_name"] = "images/" + img["file_name"].split("-")[0] + ".jpg"
    id_map[image_id] = i 
    images.append(img)

annotations = []
for ann in det_json["annotations"]:
    i = id_map[ann["image_id"]]
    caption = cat_map[ann["category_id"]]
    if "juice" in caption:
        caption = caption.split()[0] + " papaya juice"
    images[i]["caption"] = caption
    name = inf.singularize(" ".join(caption.split()[1:]))
    ann["category_id"] = orig_cat_map[name]
    counts[ann["category_id"]]  += 1
    annotations.append(ann)

new_images = []
for i, image in enumerate(images): 
    if image.get("caption"):
        new_images.append(image)

oracle_anns_map = defaultdict(list)
oracle_file_name_map = defaultdict(int)
oracle_id_map = defaultdict(int)

for i, img in enumerate(objdet_images):
    oracle_file_name_map[img["file_name"]] = img["id"]
    oracle_id_map[img["id"]] = i

for ann in oracle_annotations:
    oracle_anns_map[ann["image_id"]].append(ann)

for img in objdet_images:
    file_name = img["file_name"].split("/")[-1]
    img["id"] = oracle_file_name_map[file_name]
    img["file_name"] = "images/" + file_name

oracle_anns_count = defaultdict(int)
new_input_oracle_annotations = []
for img in new_images:
    file_name = img["file_name"].split("/")[-1]
    if img["id"] == 12: 
        print("test")
        print(file_name)
    oracle_anns_count[oracle_file_name_map[file_name]] += 1
    if img["id"] == 12: 
        print(len(oracle_anns_map[12]))
        print(oracle_file_name_map[file_name])
    for ann in oracle_anns_map[oracle_file_name_map[file_name]]:
        ann = ann.copy()
        ann["image_id"] = img["id"]
        ann["id"] = len(new_input_oracle_annotations)
        if img["id"] == 12: 
            print("hi")   
        new_input_oracle_annotations.append(ann)

# new_input_oracle_annotations = []

for ann in new_input_oracle_annotations: 
    caption = cat_map[ann["category_id"]]
    if "juice" in caption:
        caption = "papaya juice"
    name = inf.singularize(caption)
    ann["category_id"] = orig_cat_map[name]
    
new_annotations = {
    "images": new_images,
    "annotations": annotations, 
    "input_oracle_annotations": new_input_oracle_annotations,
    "categories": categories
}

json.dump(new_annotations, open(os.path.join(ann_dir, save_filename), "w"))

FileNotFoundError: [Errno 2] No such file or directory: '../../annotations/real/smallv1\\iccv-real-small-diffobj-objdet.json'

In [56]:
print(oracle_file_name_map["000015.jpg"])

count = 0

for img in objdet_json["images"]: 
    if img["id"] == 4: 
        print(img)
for ann in objdet_json["annotations"]: 
    if ann["image_id"] == 4: 
        print(ann)
        count += 1 
print(count)
oracle_anns_map[4]
cat_map[94]

4
{'id': 4, 'datatorch_id': 'f9a286aa-b0aa-4f29-8991-a44b28dfa6d2', 'storage_id': '1c8751ef-dfc8-4b41-9f8b-c5dd5ff755e2', 'path': '000015.jpg', 'width': 256, 'height': 256, 'file_name': 'images/000015.jpg', 'metadata': {}, 'date_captured': '2023-02-27T09:31:11.229Z'}
{'id': 81, 'datatorch_id': '47948905-da9f-4068-b4a4-4831eae2c753', 'image_id': 4, 'category_id': 94, 'segmentation': [[0.628897004223525, 83.1930397732098, 135.21846968618252, 83.1930397732098, 135.21846968618252, 171.813638687035, 0.628897004223525, 171.813638687035]], 'area': 11927.41, 'bbox': [0.628897004223525, 83.1930397732098, 134.589572681959, 88.6205989138252], 'iscrowd': 0}
{'id': 185, 'datatorch_id': 'b065f062-d723-42bb-bf85-18b3a07e86a5', 'image_id': 4, 'category_id': 101, 'segmentation': [[43.8876057238763, 94.2640833838732, 113.83627096281, 94.2640833838732, 113.83627096281, 158.525865270048, 43.8876057238763, 158.525865270048]], 'area': 4495.03, 'bbox': [43.8876057238763, 94.2640833838732, 69.9486652389337, 6

'banana'

In [57]:
len(oracle_anns_map[4])

3

In [58]:
new_images[3]

{'id': 12,
 'datatorch_id': 'b48835ba-ac32-4bf9-b696-81669cc1536d',
 'storage_id': '1c8751ef-dfc8-4b41-9f8b-c5dd5ff755e2',
 'path': '000015-00.jpg',
 'width': 256,
 'height': 256,
 'file_name': 'images/000015.jpg',
 'metadata': {},
 'date_captured': '2023-02-27T10:56:14.284Z',
 'caption': 'an onion'}

In [59]:
# for ann in annotations: 
#     if ann["image_id"] == 42: 
#         print(ann)

for ann in new_input_oracle_annotations: 
    if ann["image_id"] == 12: 
        print("hi")
        print(ann["category_id"])

hi
9
hi
1
hi
1
